In [ ]:
complement_dict = {'a':'t', 'c':'g', 'g':'c', 't':'a'}

In [ ]:
complement_dict['t']

In [ ]:
with open('F:/Datasets/CRYPTIC_DATA/Cryptic_Mutation_Counts/mutation_counts_1000_2000_.pkl', 'rb') as f:
    temp = pickle.load(f)
print(temp)

In [ ]:
for i in range(5):
    print("the desktop " + str(i)  + " is really impressive")

In [ ]:
from jupyter_dash import JupyterDash
import dash_bio as dashbio
from dash import html
import urllib.request as urlreq
from dash.dependencies import Input, Output
project_dir = 'F:/Project_Data/E_Pump_Project'
import dash

In [ ]:
app = JupyterDash(__name__)

#app = dash.Dash(__name__)

with open(project_dir + '/align_output_AL123456.3@Rv1063c', 'r') as f:
    data2 = f.read()

app.layout = html.Div([
    dashbio.AlignmentChart(
        id='my-default-alignment-viewer',
        data=data2,
        height=1800,
        tilewidth=30,
    ),
    html.Div(id='default-alignment-viewer-output')
])

@app.callback(
    Output('default-alignment-viewer-output', 'children'),
    Input('my-default-alignment-viewer', 'eventDatum')
)
def update_output(value):
    if value is None:
        return 'No data.'
    return str(value)

if __name__ == '__main__':
    #app.run_server(mode='jupyterlab', debug=False)
    #app.run_server("external")
    app.run_server(debug=False)

In [ ]:
with open(project_dir + '/align_output_NC_000962.3@Rv2846c', 'r') as f:
    data2 = f.read()


In [1]:
import pandas as pd
import pickle

In [2]:
project_dir = 'F:/Project_Data/Project_10'
datasets_dir = 'F:/Datasets/CRYPTIC_DATA'

In [ ]:
variant_df = pd.read_csv(datasets_dir + "/VARIANTS.csv") 
variant_df_2 = variant_df[['UNIQUEID', 'VARIANT', 'MUTATION_TYPE', 'IS_NULL', 'IS_HET', 'IS_FILTER_PASS', 'IS_SNP', 'REF', 'ALT', 'FRS', 'GT_CONF_PERCENTILE', 'DP', 'GENOME_INDEX']]    
variant_df = []
with open(project_dir + '/variant_df_2.pkl', 'wb') as f:
    pickle.dump(variant_df_2, f)  
print("Stage_1_Complete")

In [ ]:
with open(project_dir + '/variant_df_2.pkl', 'rb') as f:
    variant_df_2 = pickle.load(f)  
print(len(variant_df_2))
genomes_df = pd.read_csv(datasets_dir + '/GENOMES.csv')
gpi_genomes_df = genomes_df[['UNIQUEID','BELONGS_GPI']] 
print(len(gpi_genomes_df))
gpi_variant_df = pd.merge(variant_df_2, gpi_genomes_df, how='left', on = 'UNIQUEID')
variant_df_2 = []
with open(project_dir + '/nick_variant_df.pkl', 'wb') as f:
    pickle.dump(gpi_variant_df, f)    
print("Stage_2_Complete")

In [39]:
with open(project_dir + '/nick_variant_df.pkl', 'rb') as f:
    variant_df = pickle.load(f)  

In [40]:
gpi_variant_df = variant_df[variant_df['BELONGS_GPI'] == True]

In [41]:
for i, r in gpi_variant_df.iterrows():
    if r['FRS'] >= 0.9 and r['DP'] >= 5 and r['GT_CONF_PERCENTILE'] >= 0.5:
        gpi_variant_df.at[i, 'CLOCK_PASS'] = True
    else:
        gpi_variant_df.at[i, 'CLOCK_PASS'] = False

C:\Users\nicho\AppData\Local\Temp/ipykernel_26776/3007769752.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpi_variant_df.at[i, 'CLOCK_PASS'] = True


In [42]:
gpi_variant_df.groupby(['CLOCK_PASS']).size().reset_index().rename(columns={0:'count'})

,CLOCK_PASS,count
0,False,1552144
1,True,16607233


In [43]:
filter_fail_sample = gpi_variant_df.groupby(['CLOCK_PASS', 'GENOME_INDEX']).size().reset_index().rename(columns={0:'count'})

In [44]:
a = filter_fail_sample[filter_fail_sample['CLOCK_PASS'] == True]
b = filter_fail_sample[filter_fail_sample['CLOCK_PASS'] == False]

In [45]:
c = pd.merge(a, b, how = 'outer', on = 'GENOME_INDEX')
c['ratio'] = c['count_x']/(c['count_x'] + c['count_y'])

In [46]:
c[c['ratio'] <0.9]

,CLOCK_PASS_x,GENOME_INDEX,count_x,CLOCK_PASS_y,count_y,ratio
90,True,506,1.0,False,1.0,0.500000
265,True,1377,1.0,False,1.0,0.500000
309,True,1549,4.0,False,1.0,0.800000
315,True,1591,152.0,False,46.0,0.767677
512,True,2422,12.0,False,2.0,0.857143
...,...,...,...,...,...,...
528946,True,4411307,1.0,False,1.0,0.500000
528949,True,4411317,1.0,False,1.0,0.500000
528993,True,4411455,1.0,False,1.0,0.500000
529000,True,4411478,1.0,False,1.0,0.500000


In [47]:
c[c['CLOCK_PASS_x'].isna()]

,CLOCK_PASS_x,GENOME_INDEX,count_x,CLOCK_PASS_y,count_y,ratio
529004,NaN,1165,NaN,False,1.0,NaN
529005,NaN,1566,NaN,False,1.0,NaN
529006,NaN,1611,NaN,False,1.0,NaN
529007,NaN,1640,NaN,False,1.0,NaN
529008,NaN,2495,NaN,False,1.0,NaN
...,...,...,...,...,...,...
553141,NaN,4410066,NaN,False,1.0,NaN
553142,NaN,4410663,NaN,False,1.0,NaN
553143,NaN,4410754,NaN,False,1.0,NaN
553144,NaN,4411436,NaN,False,1.0,NaN


In [ ]:
compass_df = pd.read_csv('F:/Datasets/CRYPTIC_DATA/COMPASS.csv', names = ['species', 'start', 'stop','count'])
compass_dict = {}
for i, r in compass_df.iterrows():
    for k in range(r['start'], r['stop']):     #  Should te start-1 but seems to ve inconsistency with mutations!!
        compass_dict[k] = 'Y'